In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir("gdrive/MyDrive/Colab Notebooks/DeepLearning2/4.DeepNeuralNetwork")

Mounted at /content/gdrive


# 1. Huấn luyện model

Trong bước này chúng ta sẽ huấn luyện một model phân loại trên bộ dữ liệu iris và dự báo chúng dựa trên dữ liệu đầu vào.

In [17]:
# Đọc dữ liệu
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Phân chia tập train/test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) 
print('There are {} samples in the training set and {} samples in the test set'.format(X_train.shape[0], X_test.shape[0]))

# Chuẩn hóa dữ liệu
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.transform(X_test)

# Hàm save và load file
import pickle
def _save_pkl(path, obj):
  with open(path, 'wb') as f:
    pickle.dump(obj, f)

def _load_pkl(path):
  with open(path, 'rb') as f:
    obj = pickle.load(f)
  return obj

# Huấn luyện mô hình
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5, weights='distance')
knn.fit(X_train_scl, y_train)

# Kiểm tra độ chính xác model trên tập train và test
from sklearn.metrics import accuracy_score
y_pred_train = knn.predict(X_train_scl)
y_pred_test = knn.predict(X_test_scl)
print('accuracy on train: ', accuracy_score(y_pred_train, y_train))
print('accuracy on test: ', accuracy_score(y_pred_test, y_test))

# Save model
_save_pkl('knn.pkl', knn)
_save_pkl('scaler.pkl', scaler)

There are 105 samples in the training set and 45 samples in the test set
accuracy on train:  1.0
accuracy on test:  0.9777777777777777


In [14]:
knn.predict_proba([[6.4, 3.2, 5.3, 2.3]])

array([[0., 0., 1.]])

Huấn luyện raw data, không sử dụng scaler

In [4]:
# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors = 5, weights='distance')
# knn.fit(X_train, y_train)

KNeighborsClassifier(weights='distance')

In [5]:
# # Kiểm tra accuracy trên x_train và x_test
# from sklearn.metrics import accuracy_score
# y_train_pred = knn.predict(X_train)
# y_test_pred = knn.predict(X_test)

# accuracy_score(y_train_pred, y_train)
# accuracy_score(y_test_pred, y_test)

0.9777777777777777

In [6]:
# _save_pkl('knn_no_scale.pkl', knn)

Dự báo

In [18]:
import pickle

def _load_pkl(path):
  with open(path, 'rb') as f:
    obj = pickle.load(f)
  return obj

knn = _load_pkl('knn.pkl')
scaler = _load_pkl('scaler.pkl')

In [8]:
import numpy as np
# X_input = np.array([[4,5,3,3.5]])
# X_std = scaler.transform(X_input)
# knn.predict(X_std)

# 2. Xây dựng API

## 2.1. API là gì ?

API được coi như chất liệu để làm nên một website. Khi chúng ta tương tác với một ứng dụng là tương tác với dữ liệu của ứng dụng đó được render trong một template html. Dữ liệu đó đến từ đâu? Nó được truyền đến thiết bị của bạn thông qua chính API. API là chiếc cầu nối dữ liệu qua lại giữa client và server và thay đổi những gì mà chúng ta nhìn thấy trên front end.

![](https://viblo.asia/uploads/44302202-a38e-447d-8c92-6f511371e37e.png)

Các phương thức tương tác dữ liệu trên API chính bao gồm: `GET, POST, PUT, PATCH, DELETE`. Đối với những beginner thì mình nghĩ chỉ cần hiểu GET, POST là đủ.

GET: Client nhận dữ liệu từ server.

POST: Client gửi dữ liệu lên server. Server xử lý dữ liệu và trả về một kết quả.

## 2.2. Khởi tạo API trên flask

Để khởi tạo các API trên flask chúng ta sẽ phải khởi tạo app trước. Sau đó tương ứng với mỗi API chúng ta sẽ khai báo 3 thành phần:

* route: Địa chỉ url của api.
* method: Phương thức để tương tác với api. VD: Nếu bạn muốn gửi dữ liệu lên server thi là POST, muốn nhận dữ liệu từ server thì là GET.
* hàm xử lý dữ liệu: Quyết định dữ liệu sẽ được xử lý như thế nào và trả ra cho client những gì?

Cùng tìm hiểu về khởi tạo một api flask qua ví dụ đơn giản sau :

In [9]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [10]:
# Encoding numpy to json
import json
class NumpyEncoder(json.JSONEncoder):
    '''
    Encoding numpy into json
    '''
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, np.int32):
            return int(obj)
        if isinstance(obj, np.int64):
            return int(obj)
        if isinstance(obj, np.float32):
            return float(obj)
        if isinstance(obj, np.float64):
            return float(obj)
        return json.JSONEncoder.default(self, obj)

In [19]:
from PIL import Image
import numpy as np
from flask import Flask, request
import flask
import json

# Khởi tạo model.
global model 
model = None
# Khởi tạo flask app
app = Flask(__name__)

# Khai báo các route 1 cho API
@app.route("/", methods=["GET"])
# Khai báo hàm xử lý dữ liệu.
def _hello_world():
  return "Hello world"

# Khai báo các route 2 cho API
@app.route("/predict", methods=["POST"])
# Khai báo hàm xử lý dữ liệu.
def _predict():
  data = {"success": False}
  request_body = request.json()
  if request_body:
    # Lấy sepal_length
    sepal_length = request_body['sp_len']
    # Lấy sepal_width
    sepal_width = request_body['sp_wid']
    # Lấy petal_length
    petal_length = request_body['pen_len']
    # Lấy petal_width
    petal_width = request_body['pen_wid']
    # Convert sang numpy array input
    X_input = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
    # Dự báo nhãn và xác suất.
    label = model.predict(X_input)
    # Dự báo phân phối xác suất
    dist_probs = model.predict_proba(X_input)
    # Truyền vào data form response
    data["probability"] = dist_probs
    data["success"] = True
    return json.dumps(data, ensure_ascii=False, cls=NumpyEncoder)

if __name__ == "__main__":
  print("App run!")
  # Load model và scaler
  model = _load_pkl('knn.pkl')
  scaler = _load_pkl('scaler.pkl')
  app.run(debug=False, host='localhost', threaded=False)

App run!
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)


Nhìn code trên bạn thấy gì?

Bỏ qua các tiểu tiết xử lý bên trong. Nhìn khải quát các hàm thì ta có thể thấy đây chính là file thiết kế các API của project này. Đồng thời ta cũng nhận định đây là file execute chính của project vì nó có điều kiện __main__ ở cuối.

Cụ thể hơn trong file này các bạn có thể thấy chúng ta thực hiện các chức năng sau:

* Khởi tạo Flask application: dòng `app = Flask(__name__)`.
* Khai báo các đường link của api thông qua các câu lệnh @app.route(). Nếu các bạn hiểu sâu hơn thì đây chính là decorator trong python có tác dụng bổ sung thêm chức năng cho hàm bên dưới.
* Các hàm xử lý dữ liệu của API như `_predict(), _hello_world()`.
* Chúng ta cần chú ý tới hàm số `_predict()`. Bên trong hàm này ta sẽ nhận file dữ liệu ảnh được gửi lên từ client, sau đó parse sang định dạng numpy array. Sử dụng model được khai báo global để dự báo kết quả. Chi tiết chức năng của từng lệnh mình đã comment trong file.

Dành cho bạn nào là begginer: Khi chúng ta chạy file bằng câu lệnh:

`python server.py`

Chương trình sẽ thực thi các lệnh trong điều kiện if __name__ == "__main__": Đầu tiên. Do đó model sẽ được load đầu tiên. Do ở đây mình không dùng lệnh async await nên chương trình sẽ chờ đến khi load model xong thì mới chạy lệnh bên dưới. Python sẽ hơi khác so với các ngôn ngữ compile như java hay C, C++ một chút bởi vì nó mặc định là xử lý đơn luồng nên sẽ chạy tuần tự từ trên xuống dưới. Ngoài python thì còn một ngôn ngữ rất nổi tiếng khác cũng chạy đơn luồng, đó là javascript.

Sau khi load xong model thì chương trình sẽ start app thông qua lệnh `app.run()`. Lúc này bạn đã có thể tương tác được với các api của nó.

# 3. Tạo giao diện

Giao diện hay còn gọi là frontend là những phần mà người dùng được phép nhìn thấy của một website/app. Người dùng tương tác với website hoặc app sẽ thông qua giao diện. Định dạng của giao diện là một `html` file. Đây là một siêu văn bản cho phép trình bày các nội dung như nút ấn, tiêu đề, input box, hình ảnh,... dưới dạng biểu mẫu cho trước.

Để  khởi tạo giao diện chúng ta cần làm quen với các thẻ trong [html](https://www.w3schools.com/html/). Nội dung của một số thẻ chính trong html:

* div: Tạo ra các vùng division trong một thẻ html
* section: Tạo ra các section trong một thể html. Có chức năng gần tương tự như div.
* a: Đánh dấu nội dung văn bản là article để google đọc và quest content dễ dàng hơn.
* p: paragraph, là nội dung đánh dấu đoạn văn bản.
* img: thẻ chứa hình ảnh.
* input: Thẻ nhập nội dung.
* label: Thẻ nhập nội dung.
* button: Nút ấn.
* ui: Đánh dấu các item cho list nhưng không có số thứ tự.
* li: Đánh dấu các item cho list nhưng có số thứ tự.
* h1, h2, h3,...: các header.

Bên dưới chúng ta sẽ tạo ra 4 input text điền các giá trị tương ứng với 4 biến đầu vào và một nút ấn để POST dữ liệu lên server. Để giao diện có thể truy cập được một cách công khai thì 

Bên dưới là file templates/index.html

```
<!DOCTYPE html>
<html>
<head>
<title>Iris classifier</title>
<link href='https://fonts.googleapis.com/css?family=Open+Sans:400,700' rel='stylesheet' type='text/css'>
<link href='https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css' rel='stylesheet' type='text/css'>
<link rel="stylesheet" href= "{{ url_for('static',filename='app.css') }}">
<link rel="stylesheet" href= "../static/app.css">
<script src="https://code.jquery.com/jquery-3.1.1.min.js"></script>
</head>

<body>
<div id="maintable">
    <h1>Classifier Iris<h1>
    <form id="sub_form" action="/predict" method="POST">
        <label >Sepal Length:</label>
        <input type="text" id="seq_len" name="sp_len"><br><br>
        <label >Sepal Width:</label>
        <input type="text" id="seq_wid" name="sp_wid"><br><br>
        <label >Pental Length:</label>
        <input type="text" id="pen_len" name="pen_len"><br><br>
        <label >Pental Width:</label>
        <input type="text" id="pen_wid" name="pen_wid"><br><br>
        <input type="submit" value="Submit">
    </form>
    <br/>
    <div id='id_result' class="col" style="display:None">
        <h1>Predict Result: </h1>
        <div id='id_class'>Class</div>
        <div id='id_prob'>Probability</div>
    </div>
</body>

<script type="text/javascript">
// var dataJson = JSON.stringify(newData);
$('#sub_form').submit(function(e){
    var serial_data = $(this).serializeArray();
    var data = {};
    for (i = 0; i < serial_data.length; i++){
        var el = serial_data[i];
        if (el['value'] == ""){
            data[el['name']] = parseFloat(0);
        } else {
            data[el['name']] = parseFloat(el['value']);
        }
    };
    console.log(data);
    dataJson = JSON.stringify(data);
    e.preventDefault();
    var req = $.ajax({
        url: $(this).attr('action'),
        type: 'POST',
        dataType:"json",
        contentType:"application/json",
        data: dataJson,
        cache: false,
        contentType: false,
        processData: false,
        success: function(data){
                console.log("Submit was succesful!");
                let div_result = document.getElementById('id_result');
                let class_label = document.getElementById('id_class');
                let prob = document.getElementById('id_prob');
                div_result.style.display='table-cell';
                class_label.innerText = "Class: " + data['label'];
                prob.innerText = "Probability: " + 100*(data['probability'][0]) + "%";
                console.log(data['label'], data['probability'][0]);
            },
        error: function(jqXHR, textStatus, errorMessage){
            console.log('An error occurred.');
            console.log(errorMessage);
        }
    })
})
</script>
</html>
```



# 4. Deploy model trên heroku app.

Bước 1: Đầu tiên chúng ta cần tạo môi trường ảo mà trong đó đóng gói tất cả các packages sử dụng cho mô hình dự báo. Khi deploy trên heroku thì website sẽ được vận hành trên môi trường ảo đó.

`pipenv install flask gunicorn numpy sklearn pickle`

Nếu chưa có pipenv package thì có thể cài qua lệnh

`pip3 install pipenv`

Bước 2: Tạo Procfile

`touch Procfile`

Procfile là file thực thi của heroku khi app được lauch. Bên trong Procfile chúng ta khai báo :

`web: gunicorn app:server`

`heroku ps:scale web=1`

Lệnh trên sẽ cho chúng ta biết loại server mà chúng ta sử dụng là gunicorn, file để kích hoạt app là server.py.

Bước 3: Tạo file `runtime.txt` có nội dung

`python-3.7.2`

File này sẽ khai báo môi trường chạy ứng dụng sử dụng python version 3.6.0.

Bước 4: Khởi động môi trường ảo ở local và test thử app.

`pipenv shell`

Môi trường ảo tại local sẽ được migrate lên heroku nên chúng ta sẽ test trước trên môi trường ảo tại local trước khi launch app.

Bước 5: Lauch app trên heroku.
- Download heroku cli
- Khởi tạo app bằng lệnh 
`heroku create app_name`
- Deploy:
`git push heroku master`

https://devcenter.heroku.com/articles/getting-started-with-python

https://www.geeksforgeeks.org/deploy-python-flask-app-on-heroku/
